<a href="https://colab.research.google.com/github/AARYAK12345/fakenewsdetection/blob/main/Copy_of_Fake_news_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer #stemming returns root word
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
# link to ds https://www.kaggle.com/c/fake-news/data?select=train.csv


In [ ]:
import nltk
nltk.download('stopwords')
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
data=pd.read_csv('train.csv')
print(data)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [ ]:
data.shape

(20800, 5)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [ ]:
data.head()

id                                              title              author  \
0   0  House Dem Aide: We Didn’t Even See Comey’s Let...       Darrell Lucus   
1   1  FLYNN: Hillary Clinton, Big Woman on Campus - ...     Daniel J. Flynn   
2   2                  Why the Truth Might Get You Fired  Consortiumnews.com   
3   3  15 Civilians Killed In Single US Airstrike Hav...     Jessica Purkiss   
4   4  Iranian woman jailed for fictional unpublished...      Howard Portnoy   

                                                text  label  
0  House Dem Aide: We Didn’t Even See Comey’s Let...      1  
1  Ever get the feeling your life circles the rou...      0  
2  Why the Truth Might Get You Fired October 29, ...      1  
3  Videos 15 Civilians Killed In Single US Airstr...      1  
4  Print \nAn Iranian woman has been sentenced to...      1

In [ ]:
data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
data.fillna('')

id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiumnews.com   
3                                Jessica Purkiss   
4                                 Howard Portnoy   
...                                          ...   
20795                              Jerome Hudson   
20796                           Benjamin Hoffman   
20797  Michael J. de la Merced and Rachel Abrams   
20798                                Alex Ansary   
20799                              David Swanson   

                                                    text  label  
0      House Dem Aide: We Didn’t Even See Comey’s Let...      1  
1      Ever get the feeling your life circles the rou...      0  
2      Why the Truth Might Get You Fired October 29, ...      1  
3      Videos 15 Civilians Killed In Single US Airstr...      1  
4      Print \nAn Iranian woman has been sentenced to...      1  
...                                                  ...    ...  
20795  Rapper T. I. unloaded on black celebrities who...      0  
20796  When the Green Bay Packers lost to the Washing...      0  
20797  The Macy’s of today grew from the union of sev...      0  
20798  NATO, Russia To Hold Parallel Exercises In Bal...      1  
20799    David Swanson is an author, activist, journa...      1  

[20800 rows x 5 columns]

In [ ]:
data.isnull().sum()


id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
#using title and author to predict
data['content']=data['author']+' '+data['title']

In [ ]:
print(data)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [ ]:
X=data.drop(columns='label',axis=1)

In [ ]:
Y=data['label']

id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiumnews.com   
3                                Jessica Purkiss   
4                                 Howard Portnoy   
...                                          ...   
20795                              Jerome Hudson   
20796                           Benjamin Hoffman   
20797  Michael J. de la Merced and Rachel Abrams   
20798                                Alex Ansary   
20799                              David Swanson   

                                                    text  \
0      House Dem Aide: We Didn’t Even See Comey’s Let...   
1      Ever get the feeling your life circles the rou...   
2      Why the Truth Might Get You Fired October 29, ...   
3      Videos 15 Civilians Killed In Single US Airstr...   
4      Print \nAn Iranian woman has been sentenced to...   
...                                                  ...   
20795  Rapper T. I. unloaded on black celebrities who...   
20796  When the Green Bay Packers lost to the Washing...   
20797  The Macy’s of today grew from the union of sev...   
20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799    David Swanson is an author, activist, journa...   

                                                 content  
0      Darrell Lucus House Dem Aide: We Didn’t Even S...  
1      Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...  
2      Consortiumnews.com Why the Truth Might Get You...  
3      Jessica Purkiss 15 Civilians Killed In Single ...  
4      Howard Portnoy Iranian woman jailed for fictio...  
...                                                  ...  
20795  Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...  
20796  Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...  
20797  Michael J. de la Merced and Rachel Abrams Macy...  
20798  Alex Ansary NATO, Russia To Hold Parallel Exer...  
20799            David Swanson What Keeps the F-35 Alive  

[20800 rows x 5 columns]

In [ ]:
port_stem=PorterStemmer()

In [ ]:
def stemming(content):
    # Check if content is a string
    if isinstance(content, str):
        # Remove non-alphabetical characters
        stemmed_content = re.sub('[^a-zA-Z]'," ", content)

        # Convert to lowercase
        stemmed_content = stemmed_content.lower()

        # Tokenize into words
        stemmed_content = stemmed_content.split()

        # Stem words and remove stopwords
        stemmed_content = [port_stem.stem(word) for word in stemmed_content if word not in stopwords.words('english')]

        # Join the words back into a single string
        stemmed_content = ' '.join(stemmed_content)

        return stemmed_content
    else:
        # Handle non-string values, e.g., return an empty string
        return ''

In [ ]:
data['content']=data['content'].apply(stemming)


In [ ]:
data['content']

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object

In [ ]:
X=data['content'].values
Y=data['label'].values

In [ ]:
X.shape

(20800,)

In [ ]:
vectorizere=TfidfVectorizer()
vectorizere.fit(X)
X=vectorizere.transform(X) #to transform text into features

In [ ]:
print(X)

  (0, 14626)	0.28553358349485225
  (0, 12567)	0.2563589820798857
  (0, 8310)	0.3596536533885757
  (0, 8048)	0.2934713957861362
  (0, 7190)	0.24644399370485162
  (0, 6552)	0.21885788468666617
  (0, 4637)	0.23132772852542255
  (0, 3543)	0.2689082732577136
  (0, 3359)	0.3596536533885757
  (0, 2757)	0.24749629983427848
  (0, 2312)	0.37305738773975167
  (0, 247)	0.2702588721108487
  (1, 15663)	0.30543189988638736
  (1, 6377)	0.19530026425832656
  (1, 5140)	0.7101106653194483
  (1, 3328)	0.2633913741576576
  (1, 2619)	0.19610930415709218
  (1, 2066)	0.38047138877486064
  (1, 1764)	0.15430291936631466
  (1, 1391)	0.29649664509588236
  (2, 14560)	0.41792081528179686
  (2, 8973)	0.4931379270922205
  (2, 5579)	0.35039256210774505
  (2, 5031)	0.3876367834516749
  (2, 2895)	0.4571578367807032
  :	:
  (20797, 12239)	0.25299971331630106
  (20797, 11515)	0.2746816397264756
  (20797, 11321)	0.2462639038918503
  (20797, 9605)	0.08009780291419748
  (20797, 8942)	0.17302467550398123
  (20797, 8879)	0.292

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=3)

In [ ]:
model=LogisticRegression()

In [ ]:
model.fit(X_train,Y_train)

LogisticRegression()

In [ ]:
model.score(X_train,Y_train)

0.9857572115384615

In [ ]:
model.score(X_test,Y_test)

0.9778846153846154

In [ ]:
#building a prediction system
X_new=X_test[21]
prediction=model.predict(X_new)
print(prediction)
if (prediction[0]==0):
    print('The news is real')
else:
    print('The news is fake')

[1]
The news is fake
